In [ ]:
import requests
import random
import json
from hashlib import md5
from docx import Document
from docx.shared import Pt
import time
import re

# Set your own baidu appid/appkey.
appid = 'XXXXXX'
appkey = 'XXXXXX'

endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path

file_path = "document.docx"
output_path = "translated_document.docx"

jump_text=['测试描述Test Description',
           '前提条件 Prerequisites',
          '测试目的Test Purpose',
          '前提条件 Prerequisites',
          '测试过程Test Procedure',
          '测试步骤Test Steps',
          '测试结果Test Result',
          '测试LOG：Test LOG:',
          '测试图片Test Pictures']

In [101]:

# Generate salt and sign
def make_md5(s, encoding='utf-8'):
    return md5(s.encode(encoding)).hexdigest()


def translate_text(text, from_lang="zh", to_lang="en"):
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + text + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': text, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    
    # Send request
    retry_cnt=10
    while(retry_cnt):
        try:
            r = requests.post(url, params=payload, headers=headers)
            result = r.json()
            data = json.loads(json.dumps(result, indent=4, ensure_ascii=False))
            time.sleep(0.08)
            break
        except:
            retry_cnt -= 1
            time.sleep(0.5)
    # Show response
    #print(json.dumps(result, indent=4, ensure_ascii=False))
    return data["trans_result"][0]['dst']



def has_chinese(text):
    pattern = re.compile(r'[\u4e00-\u9fff]')
    return bool(pattern.search(text))

# def copy_paragraph_format(source_para, target_para):
#     # 复制基本的段落格式
#     target_para.alignment = source_para.alignment
#     target_para.paragraph_format.left_indent = source_para.paragraph_format.left_indent
#     target_para.paragraph_format.right_indent = source_para.paragraph_format.right_indent
#     target_para.paragraph_format.space_before = source_para.paragraph_format.space_before
#     target_para.paragraph_format.space_after = source_para.paragraph_format.space_after
#     target_para.paragraph_format.line_spacing = source_para.paragraph_format.line_spacing



# 加载Word文档
doc = Document(file_path)
translated_doc = Document()

#增加进度条
doc_len=(len(doc.paragraphs) + len(doc.tables))
spin=['--','\\','|','/']
i=0

# 遍历文档中的每个段落
for para in doc.paragraphs:
    # 翻译文本
    if para.text.strip():  # 确保不翻译空段落
        if has_chinese(para.text):
            translated_text = translate_text(para.text)
            para.add_run("\n" + translated_text)
        else:
            continue
        #打印进度条
    i +=1
    print('已完成：%d %% %s \r'%(round(i/doc_len*100,2),spin[i%len(spin)]),end="")


for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            cell_text = ' '.join(cell.text for cell in cell.paragraphs)
            if cell_text.strip():
                if cell_text not in jump_text and has_chinese(cell_text):
                    translated_text = translate_text(cell_text)
                    cell.add_paragraph("\n" + translated_text)
                else:
                    continue
    #打印进度条
    i +=1
    print('已完成：%d %% %s \r'%(round(i/doc_len*100,2),spin[i%len(spin)]),end="")

# 保存修改后的文档
print('已完成文档翻译！')
doc.save(output_path)



KeyboardInterrupt: 

In [99]:
doc.save(output_path)

In [30]:
## 百度api  test
import requests
import random
import json
from hashlib import md5

# Set your own appid/appkey.
appid = '20240510002047694'
appkey = 'C5OPEMo9YAAG56LmGr2w'

# For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
from_lang = 'en'
to_lang =  'zh'

endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path

query = 'Hello World! This is 1st paragraph.\nThis is 2nd paragraph.'

# Generate salt and sign
def make_md5(s, encoding='utf-8'):
    return md5(s.encode(encoding)).hexdigest()

salt = random.randint(32768, 65536)
sign = make_md5(appid + query + str(salt) + appkey)

# Build request
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}

# Send request
r = requests.post(url, params=payload, headers=headers)
result = r.json()

# Show response
print(json.dumps(result, indent=4, ensure_ascii=False))


{
    "from": "en",
    "to": "zh",
    "trans_result": [
        {
            "src": "Hello World! This is 1st paragraph.",
            "dst": "你好，世界！这是第一段。"
        },
        {
            "src": "This is 2nd paragraph.",
            "dst": "这是第2段。"
        }
    ]
}
